In [1]:
import pandas as pd

import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import multilabel_confusion_matrix
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from tqdm import tqdm, trange
from transformers import AdamW
from torch.nn import BCEWithLogitsLoss, BCELoss
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


# Data

## Reading the data

In [2]:
df = pd.read_csv('data/OpenI/OpenI_cheXpertLabels.csv')
print("The total No. of rows:", len(df))
df.head()

The total No. of rows: 2447


,fileNo,COMPARISON,INDICATION,FINDINGS,IMPRESSION,expert_labels,No Finding,Cardiomegaly,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Fracture,SupportDevices
0,881,None available.,XXXX-year-old XXXX with dyspnea.,The lungs are without focal air space opacity....,No acute cardiopulmonary abnormality.,['No Finding'],1,0,0,0,0,0,0,0,0,0,0
1,1734,NaN,Back pain,Heart size and mediastinal contour are normal....,No acute cardiopulmonary process.,['No Finding'],1,0,0,0,0,0,0,0,0,0,0
2,306,NaN,NaN,The lungs are clear. Heart size is normal. No ...,Clear lungs. No acute cardiopulmonary abnormal...,['No Finding'],1,0,0,0,0,0,0,0,0,0,0
3,1951,NaN,NaN,Cardiomediastinal silhouette is normal. Pulmon...,No acute cardiopulmonary disease.,['No Finding'],1,0,0,0,0,0,0,0,0,0,0
4,1005,NaN,Pruritic.,Cardiac and mediastinal contours are within no...,No acute findings.,['No Finding'],1,0,0,0,0,0,0,0,0,0,0


In [3]:
cols = df.columns
labels = list(cols[6:])
print('Count of 1 per label: \n', df[labels].sum(), '\n') # Label counts, may need to downsample or upsample

Count of 1 per label: 
 No Finding          1391
Cardiomegaly         375
Lung Opacity         406
Edema                 46
Consolidation         30
Pneumonia             42
Atelectasis          332
Pneumothorax          23
Pleural Effusion     161
Fracture              84
SupportDevices       134
dtype: int64 



## preprocessing the data

1. removing the rows without any Impression and Finding

In [4]:
print('No. of rows with Finding:', len(df[df['FINDINGS'].notnull()]))
print('No. of rows with Impression:', len(df[df['IMPRESSION'].notnull()]))
print('No. of rows with Impression or Finding:', 
      len(df[df['IMPRESSION'].notnull() | df['FINDINGS'].notnull()]))
print('No. of rows without Impression and Finding:', 
      len(df[df['IMPRESSION'].isna() & df['FINDINGS'].isna()]))

No. of rows with Finding: 2100
No. of rows with Impression: 2415
No. of rows with Impression or Finding: 2419
No. of rows without Impression and Finding: 28


In [5]:
idx = df[df['IMPRESSION'].isna() & df['FINDINGS'].isna()].index
df = df.drop(idx)
print('No. of rows without Impression and Finding:', 
      len(df[df['IMPRESSION'].isna() & df['FINDINGS'].isna()]))

No. of rows without Impression and Finding: 0


2. Converting the labels to a single list

In [6]:
labels = ['No Finding', 'Cardiomegaly','Lung Opacity','Edema','Consolidation','Pneumonia','Atelectasis','Pneumothorax','Pleural Effusion','Fracture','SupportDevices']

df_cls = pd.DataFrame(columns = ['text', 'labels'])

def concat_cols(impression, findings):
    if impression is np.nan:
        return findings
    elif findings is np.nan:
        return impression
    else:
        return findings+impression

# create the text column:
df_cls['text'] = df.apply(lambda row: concat_cols(row['IMPRESSION'], row['FINDINGS']), axis=1)
df_cls['labels'] = df.apply(lambda row: row[labels].to_list(), axis=1) #.to_list() , .values

df_cls.head(10)

,text,labels
0,The lungs are without focal air space opacity....,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,Heart size and mediastinal contour are normal....,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,The lungs are clear. Heart size is normal. No ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,Cardiomediastinal silhouette is normal. Pulmon...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,Cardiac and mediastinal contours are within no...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5,Mediastinal contours are normal. Lungs are cle...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6,Heart size and mediastinal contours are unrema...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,"The heart is again enlarged, stable. The left ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
8,Normal heart size. No focal air space consolid...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
9,Heart size within normal limits. No focal airs...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


3. Removing the duplicate reports

In [7]:
print('Unique texts: ', df_cls.text.nunique() == df_cls.shape[0])

Unique texts:  False


In [8]:
print("Length of whole dataframe:", len(df_cls))
print("No. of unique reports:", df_cls.text.nunique())

Length of whole dataframe: 2419
No. of unique reports: 1742


In [9]:
#let's take a look at what are the duplicated texts looks like:
df_cls.text.value_counts()

The heart is normal in size. The mediastinum is unremarkable. The lungs are clear.No acute disease.                                                                                                                                                                                                                                                                                                                     49
Heart size normal. Lungs are clear. XXXX are normal. No pneumonia, effusions, edema, pneumothorax, adenopathy, nodules or masses.Normal chest                                                                                                                                                                                                                                                                           35
Both lungs are clear and expanded. Heart and mediastinum normal.No active disease.                                                                                                

In [10]:
# remove the duplicates
df_cls.drop_duplicates('text', inplace=True)

In [11]:
print('Unique texts: ', df_cls.text.nunique() == df_cls.shape[0])

Unique texts:  True


## Splitting the data
**First Attempt:**   
train_test_split from sklearn (stratified)   
Failed    

**Second Attempt**      
http://scikit.ml/stratification.html     
This didn't work, It only accepted their own type of input and didn't update the repo for the last two years.    
Failed

**Third Attempt**
splitting a multilabel dataset(stratified)     
https://vict0rs.ch/2018/05/24/sample-multilabel-dataset/    
Passed (The code of Experiment 1 is using this attempt)

**Forth Attempt**      
Splitting in a random way using sklearn    
Passed


In [12]:
#----------first attempt-------------(unsuccessful)

# from sklearn.model_selection import train_test_split

# trainX, valX, trainY, valY = train_test_split(df_cls['text'].values.tolist(), 
#                                               df_cls['labels'].values.tolist(), 
#                                               test_size=0.5,
#                                               stratify = df_cls['labels'].values.tolist())
# train, test = train_test_split(df, test_size=0.5, random_state=0, 
#                                stratify = df[labels])
# ValueError: The least populated class in y has only 1 member, which is too few. 
# The minimum number of groups for any class cannot be less than 2.

# This because of the nature of stratification. The stratify parameter set it 
# to split data in a way to allocate test_size amount of data to each class. 
# In this case, you don't have sufficient class labels of one of your classes 
# to keep the data splitting ratio equal to test_size

In [13]:
#--------second attempt--------- (unsuccessful)
# https://github.com/scikit-multilearn/scikit-multilearn

# from skmultilearn.model_selection import iterative_train_test_split
# from skmultilearn.dataset import load_dataset
# from skmultilearn.model_selection.measures import get_combination_wise_output_matrix
# from collections import Counter

# X,y, _, _ = load_dataset('scene', 'undivided')
# Counter(combination for row in get_combination_wise_output_matrix(y.A, order=2) for combination in row)

# X_train, y_train, X_test, y_test = iterative_train_test_split(df_cls['text'].values, df_cls['labels'].values, test_size = 0.3)

# pd.DataFrame({
#     'train': Counter(str(combination) for row in get_combination_wise_output_matrix(y_train.A, order=2) for combination in row),
#     'test' : Counter(str(combination) for row in get_combination_wise_output_matrix(y_test.A, order=2) for combination in row)
# }).T.fillna(0.0)

In [14]:
#shuffle
df_cls = df_cls.sample(frac=1).reset_index(drop=True)

In [15]:
#---------third attempt-------------
from copy import deepcopy

def stratify(data, classes, ratios, one_hot=False):
    """Stratifying procedure.

    data is a list of lists: a list of labels, for each sample.
        Each sample's labels should be ints, if they are one-hot encoded, use one_hot=True
    
    classes is the list of classes each label can take

    ratios is a list, summing to 1, of how the dataset should be split

    """
    # one-hot decoding
    if one_hot:
        temp = [[] for _ in range(len(data))]
        indexes, values = np.where(np.array(data).astype(int) == 1)
        for k, v in zip(indexes, values):
            temp[k].append(v)
        data = temp

    # Organize data per label: for each label l, per_label_data[l] contains the list of samples
    # in data which have this label
    per_label_data = {c: set() for c in classes}
    for i, d in enumerate(data):
        for l in d:
            per_label_data[l].add(i)

    # number of samples
    size = len(data)

    # In order not to compute lengths each time, they are tracked here.
    subset_sizes = [r * size for r in ratios]
    target_subset_sizes = deepcopy(subset_sizes)
    per_label_subset_sizes = {
        c: [r * len(per_label_data[c]) for r in ratios]
        for c in classes
    }

    # For each subset we want, the set of sample-ids which should end up in it
    stratified_data_ids = [set() for _ in range(len(ratios))]

    # For each sample in the data set
    while size > 0:
        # Compute |Di|
        lengths = {
            l: len(label_data)
            for l, label_data in per_label_data.items()
        }
        try:
            # Find label of smallest |Di|
            label = min(
                {k: v for k, v in lengths.items() if v > 0}, key=lengths.get
            )
        except ValueError:
            # If the dictionary in `min` is empty we get a Value Error. 
            # This can happen if there are unlabeled samples.
            # In this case, `size` would be > 0 but only samples without label would remain.
            # "No label" could be a class in itself: it's up to you to format your data accordingly.
            break
        current_length = lengths[label]

        # For each sample with label `label`
        while per_label_data[label]:
            # Select such a sample
            current_id = per_label_data[label].pop()

            subset_sizes_for_label = per_label_subset_sizes[label]
            # Find argmax clj i.e. subset in greatest need of the current label
            largest_subsets = np.argwhere(
                subset_sizes_for_label == np.amax(subset_sizes_for_label)
            ).flatten()

            if len(largest_subsets) == 1:
                subset = largest_subsets[0]
            # If there is more than one such subset, find the one in greatest need
            # of any label
            else:
                largest_subsets = np.argwhere(
                    subset_sizes == np.amax(subset_sizes)
                ).flatten()
                if len(largest_subsets) == 1:
                    subset = largest_subsets[0]
                else:
                    # If there is more than one such subset, choose at random
                    subset = np.random.choice(largest_subsets)

            # Store the sample's id in the selected subset
            stratified_data_ids[subset].add(current_id)

            # There is one fewer sample to distribute
            size -= 1
            # The selected subset needs one fewer sample
            subset_sizes[subset] -= 1

            # In the selected subset, there is one more example for each label
            # the current sample has
            for l in data[current_id]:
                per_label_subset_sizes[l][subset] -= 1
            
            # Remove the sample from the dataset, meaning from all per_label dataset created
            for l, label_data in per_label_data.items():
                if current_id in label_data:
                    label_data.remove(current_id)

    # Create the stratified dataset as a list of subsets, each containing the orginal labels
    stratified_data_ids = [sorted(strat) for strat in stratified_data_ids]
    stratified_data = [
        [data[i] for i in strat] for strat in stratified_data_ids
    ]

    # Return both the stratified indexes, to be used to sample the `features` associated with your labels
    # And the stratified labels dataset
    return stratified_data_ids, stratified_data
stratified_data_ids, stratified_data =  stratify(data=df_cls['labels'].values, classes=[0,1], ratios=[0.6,0.2,0.2], one_hot=False)

In [16]:
train_df = df_cls.iloc[stratified_data_ids[0],:]
test_df = df_cls.iloc[stratified_data_ids[1],:]
val_df = df_cls.iloc[stratified_data_ids[2],:]

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

print('Train: ', len(train_df))
print('Test: ', len(test_df))
print('Val: ', len(val_df))

Train:  1043
Test:  350
Val:  349


In [17]:
cols = df.columns
label_cols = list(cols[6:])
num_labels = len(label_cols)

label_counts_total = np.array(df_cls.labels.to_list()).sum(axis=0)
label_counts_train = np.array(train_df.labels.to_list()).sum(axis=0)
label_counts_test = np.array(test_df.labels.to_list()).sum(axis=0)
label_counts_val = np.array(val_df.labels.to_list()).sum(axis=0)

from prettytable import PrettyTable
pretty=PrettyTable()
pretty.field_names = ['Pathology', 'total', 'train', 'test','val']
for pathology, cnt_total, cnt_train, cnt_test, cnt_val in zip(label_cols,label_counts_total, label_counts_train, label_counts_test, label_counts_val):
    pretty.add_row([pathology, cnt_total, cnt_train, cnt_test, cnt_val])
print(pretty)    

+------------------+-------+-------+------+-----+
|    Pathology     | total | train | test | val |
+------------------+-------+-------+------+-----+
|    No Finding    |  689  |  397  | 140  | 152 |
|   Cardiomegaly   |  375  |  240  |  73  |  62 |
|   Lung Opacity   |  406  |  263  |  76  |  67 |
|      Edema       |   46  |   28  |  8   |  10 |
|  Consolidation   |   30  |   22  |  3   |  5  |
|    Pneumonia     |   42  |   24  |  10  |  8  |
|   Atelectasis    |  332  |  208  |  62  |  62 |
|   Pneumothorax   |   23  |   12  |  6   |  5  |
| Pleural Effusion |  160  |   93  |  34  |  33 |
|     Fracture     |   84  |   56  |  14  |  14 |
|  SupportDevices  |  132  |   69  |  33  |  30 |
+------------------+-------+-------+------+-----+


In [18]:
#-------------4th attempt (not stratified)---------

# from sklearn.model_selection import train_test_split

# train_val_df, test_df = train_test_split(df_cls, test_size=0.2)
# train_df, val_df = train_test_split(train_val_df, test_size=0.2)

# print('Train: ', len(train_df))
# print('Test: ', len(test_df))
# print('Val: ', len(val_df))

# Save train, test, val csv files

In [42]:
train_df.to_csv('data/OpenI/cheXpertLabels/train.csv', index=False)
test_df.to_csv('data/OpenI/cheXpertLabels/test.csv', index=False)
val_df.to_csv('data/OpenI/cheXpertLabels/val.csv', index=False)

# simpletransformers implementation

* MultiLabelClassificationModel has an additional `threshold` parameter with default value 0.5
* MultiLabelClassificationModel takes in an additional optional argument pos_weight. This should be a list with the same length as the number of labels. This enables using different weights for each label when calculating loss during training and evaluation.

In [19]:
# from simpletransformers.classification import MultiLabelClassificationModel
# import pandas as pd
# import logging

# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

# # Create a MultiLabelClassificationModel
# model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=len(labels), args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 5})
# # You can set class weights by using the optional weight argument
# print(train_df.head())

# # Train the model
# model.train_model(train_df)

# # Evaluate the model
# result, model_outputs, wrong_predictions = model.eval_model(test_df)
# # print(result)
# # print(model_outputs)

# predictions, raw_outputs = model.predict(test_df['text'])
# # print(predictions)
# # print(raw_outputs)


In [20]:
# y_pred = predictions
# y_true = test_df['labels'].to_list()

In [21]:
# cms = multilabel_confusion_matrix(y_true, y_pred)
# def plot_hm(label, conMat):

#     df_cm = pd.DataFrame(conMat, range(2), range(2))
#     # plt.figure(figsize=(10,7))
#     sn.set(font_scale=1.2) # for label size
#     sn.heatmap(df_cm, annot=True, annot_kws={"size": 12}, cmap="YlGnBu", fmt="d") # font size
#     plt.show()

# for cm,label in zip(cms, labels):
#     print(label)
# #     print(cm)
#     plot_hm(label, cm)

In [22]:
# from sklearn.metrics import classification_report
# print(classification_report(y_true, y_pred))

# Hugging Face implementation - Multi label classification

This code is inspired from


post: https://towardsdatascience.com/transformers-for-multilabel-classification-71a1a0daf5e1

code: https://colab.research.google.com/github/rap12391/transformers_multilabel_toxic/blob/master/toxic_multilabel.ipynb#scrollTo=uorMX_zrnISM

In [23]:
df_cls.head()

,text,labels
0,The trachea is midline. The cardiomediastinal ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,There is hyperinflation of the lungs. A small ...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2,The heart is normal in size and contour. There...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
3,"The heart, pulmonary XXXX and mediastinum are ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,The lungs are clear. The heart pulmonary XXXX ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [24]:
print('average report length: ', round(df_cls.text.str.split().str.len().mean(),2))
print('stdev report length: ', round(df_cls.text.str.split().str.len().std(),2))

average report length:  41.17
stdev report length:  23.01


In [25]:
cols = df.columns
label_cols = list(cols[6:])
num_labels = len(label_cols)
print('Label columns: ', label_cols)

Label columns:  ['No Finding', 'Cardiomegaly', 'Lung Opacity', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Fracture', 'SupportDevices']


In [26]:
label_counts = np.array(df_cls.labels.to_list()).sum(axis=0)
for col, cnt in zip(label_cols, label_counts):
    print(f'{col}: {cnt}')

No Finding: 689
Cardiomegaly: 375
Lung Opacity: 406
Edema: 46
Consolidation: 30
Pneumonia: 42
Atelectasis: 332
Pneumothorax: 23
Pleural Effusion: 160
Fracture: 84
SupportDevices: 132


In [27]:
print('Train: ', len(train_df))
print('Test: ', len(test_df))
print('Val: ', len(val_df))

Train:  1043
Test:  350
Val:  349


## tokenization

* The input_ids are the indices corresponding to each token in our sentence.
* We can now see what the attention_mask is all about: it points out which tokens the model should pay attention to and which ones it should not (because they represent padding in this case).
* token_type_ids are for: they indicate to the model which part of the inputs correspond to the first sentence and which part corresponds to the second sentence. Note that token_type_ids are not required or handled by all models.

In [28]:
max_length = 128
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

reports_train = train_df.text.to_list()
reports_test = test_df.text.to_list()
reports_val   = val_df.text.to_list()

train = tokenizer(reports_train, padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")
test = tokenizer(reports_test, padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")
val = tokenizer(reports_val, padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")

train_labels = torch.from_numpy(np.array(train_df.labels.to_list()))
test_labels = torch.from_numpy(np.array(test_df.labels.to_list()))
val_labels = torch.from_numpy(np.array(val_df.labels.to_list()))

In [29]:
train.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [30]:
batch_size = 16
use_data_loader = True
if use_data_loader: # if the dataset is huge in size
    # Create an iterator of our data with torch DataLoader. This helps save on memory during training because, 
    # unlike a for loop, with an iterator the entire dataset does not need to be loaded into memory
    train_data = TensorDataset(train.input_ids, train.attention_mask, train_labels, train.token_type_ids)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    validation_data = TensorDataset(val.input_ids, val.attention_mask, val_labels, val.token_type_ids)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
    
    test_data = TensorDataset(test.input_ids, test.attention_mask, test_labels, test.token_type_ids)
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=validation_sampler, batch_size=batch_size)
    
else: #if the dataset is small in size
    pass

In [31]:
# Load model, the pretrained model will include a single linear classification layer on top for classification. 
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [32]:
# setting custom optimization parameters. You may implement a scheduler here as well.
# param_optimizer = list(model.named_parameters())
# no_decay = ['bias', 'gamma', 'beta']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
#      'weight_decay_rate': 0.01},
#     {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
#      'weight_decay_rate': 0.0}
# ]
# optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5,correct_bias=True)
optimizer = AdamW(model.parameters(),lr=2e-5)  # Default optimization

## Train model

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'GeForce RTX 2070 SUPER'

In [34]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):

  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()

  # Tracking variables
  tr_loss = 0 #running loss
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()

    # # Forward pass for multiclass classification
    # outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    # loss = outputs[0]
    # logits = outputs[1]

    # Forward pass for multilabel classification
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    loss_func = BCEWithLogitsLoss() 
    loss = loss_func(logits.view(-1,num_labels),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    # loss_func = BCELoss() 
    # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    train_loss_set.append(loss.item())    

    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # scheduler.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))

  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Variables to gather full output
  logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

  # Predict
  for i, batch in enumerate(validation_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    with torch.no_grad():
      # Forward pass
      outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      b_logit_pred = outs[0]
      pred_label = torch.sigmoid(b_logit_pred)

      b_logit_pred = b_logit_pred.detach().cpu().numpy()
      pred_label = pred_label.to('cpu').numpy()
      b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

  # Flatten outputs
  pred_labels = [item for sublist in pred_labels for item in sublist]
  true_labels = [item for sublist in true_labels for item in sublist]

  # Calculate Accuracy
  threshold = 0.50
  pred_bools = [pl>threshold for pl in pred_labels]
  true_bools = [tl==1 for tl in true_labels]
  val_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
  val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

  print('F1 Validation Accuracy: ', val_f1_accuracy)
  print('Flat Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Train loss: 0.42790035406748456


Epoch:  33%|███▎      | 1/3 [00:14<00:28, 14.10s/it]

F1 Validation Accuracy:  41.061946902654874
Flat Validation Accuracy:  32.95128939828081
Train loss: 0.26798246891209576


Epoch:  67%|██████▋   | 2/3 [00:28<00:14, 14.09s/it]

F1 Validation Accuracy:  75.43859649122807
Flat Validation Accuracy:  61.60458452722063
Train loss: 0.20401788525509112


Epoch: 100%|██████████| 3/3 [00:42<00:00, 14.10s/it]

F1 Validation Accuracy:  82.36775818639799
Flat Validation Accuracy:  69.34097421203438


## Test

In [35]:
# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    with torch.no_grad():
        # Forward pass
        outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        b_logit_pred = outs[0]
        pred_label = torch.sigmoid(b_logit_pred)

        b_logit_pred = b_logit_pred.detach().cpu().numpy()
        pred_label = pred_label.to('cpu').numpy()
        b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

We need to threshold our sigmoid function outputs which range from [0, 1]. Below I use 0.50 as a threshold.

In [37]:
test_label_cols = label_cols
pred_bools = [pl>0.50 for pl in pred_labels] #boolean output after thresholding

# Print and save classification report
print('Test F1 Score: ', f1_score(true_bools, pred_bools,average='micro'))
print('Test Accuracy: ', accuracy_score(true_bools, pred_bools),'\n')
clf_report = classification_report(true_bools,pred_bools,target_names=test_label_cols)
# pickle.dump(clf_report, open('classification_report.txt','wb')) #save report
print(clf_report)

Test F1 Score:  0.8095238095238095
Test Accuracy:  0.6532951289398281 

                  precision    recall  f1-score   support

      No Finding       0.96      0.96      0.96       140
    Cardiomegaly       1.00      0.77      0.87        73
    Lung Opacity       0.95      1.00      0.97        75
           Edema       0.00      0.00      0.00         8
   Consolidation       0.00      0.00      0.00         3
       Pneumonia       0.00      0.00      0.00        10
     Atelectasis       0.86      0.92      0.89        62
    Pneumothorax       0.00      0.00      0.00         6
Pleural Effusion       0.00      0.00      0.00        33
        Fracture       0.00      0.00      0.00        14
  SupportDevices       0.00      0.00      0.00        33

       micro avg       0.95      0.71      0.81       457
       macro avg       0.34      0.33      0.34       457
    weighted avg       0.73      0.71      0.71       457
     samples avg       0.81      0.75      0.77       45

/local/pkg/python/root-python-3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/local/pkg/python/root-python-3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Optimizing threshold value for micro F1 score

In [38]:
# Calculate Accuracy - maximize F1 accuracy by tuning threshold values. First with 'macro_thresholds' on the order of e^-1 then with 'micro_thresholds' on the order of e^-2

macro_thresholds = np.array(range(1,10))/10

f1_results, flat_acc_results = [], []
for th in macro_thresholds:
  pred_bools = [pl>th for pl in pred_labels]
  test_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')
  test_flat_accuracy = accuracy_score(true_bools, pred_bools)
  f1_results.append(test_f1_accuracy)
  flat_acc_results.append(test_flat_accuracy)

best_macro_th = macro_thresholds[np.argmax(f1_results)] #best macro threshold value

micro_thresholds = (np.array(range(10))/100)+best_macro_th #calculating micro threshold values

f1_results, flat_acc_results = [], []
for th in micro_thresholds:
  pred_bools = [pl>th for pl in pred_labels]
  test_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')
  test_flat_accuracy = accuracy_score(true_bools, pred_bools)
  f1_results.append(test_f1_accuracy)
  flat_acc_results.append(test_flat_accuracy)

best_f1_idx = np.argmax(f1_results) #best threshold value

# Printing and saving classification report
print('Best Threshold: ', micro_thresholds[best_f1_idx])
print('Test F1 Score: ', f1_results[best_f1_idx])
print('Test Accuracy: ', flat_acc_results[best_f1_idx], '\n')

best_pred_bools = [pl>micro_thresholds[best_f1_idx] for pl in pred_labels]
clf_report_optimized = classification_report(true_bools,best_pred_bools, target_names=label_cols)
# pickle.dump(clf_report_optimized, open('classification_report_optimized.txt','wb'))
print(clf_report_optimized)

Best Threshold:  0.46
Test F1 Score:  0.818407960199005
Test Accuracy:  0.667621776504298 

                  precision    recall  f1-score   support

      No Finding       0.96      0.97      0.97       140
    Cardiomegaly       1.00      0.81      0.89        73
    Lung Opacity       0.95      1.00      0.97        75
           Edema       0.00      0.00      0.00         8
   Consolidation       0.00      0.00      0.00         3
       Pneumonia       0.00      0.00      0.00        10
     Atelectasis       0.87      0.95      0.91        62
    Pneumothorax       0.00      0.00      0.00         6
Pleural Effusion       0.00      0.00      0.00        33
        Fracture       0.00      0.00      0.00        14
  SupportDevices       0.00      0.00      0.00        33

       micro avg       0.95      0.72      0.82       457
       macro avg       0.34      0.34      0.34       457
    weighted avg       0.73      0.72      0.72       457
     samples avg       0.82      0.7

In [ ]:
# # From Transformers: https://huggingface.co/transformers/_modules/transformers/modeling_bert.html#BertForSequenceClassification
# from torch.nn import BCEWithLogitsLoss
# class BertForMultiLabelClassification(BertPreTrainedModel):
#     def __init__(self, config):
#         super().__init__(config)
#         self.num_labels = config.num_labels

#         self.bert = BertModel(config)
#         self.dropout = nn.Dropout(config.hidden_dropout_prob)
#         self.classifier = nn.Linear(config.hidden_size, config.num_labels)

#         self.init_weights()

# [docs]
#     @add_start_docstrings_to_callable(BERT_INPUTS_DOCSTRING.format("(batch_size, sequence_length)"))
#     def forward(
#         self,
#         input_ids=None,
#         attention_mask=None,
#         token_type_ids=None,
#         position_ids=None,
#         head_mask=None,
#         inputs_embeds=None,
#         labels=None,
#     ):


#         outputs = self.bert(
#             input_ids,
#             attention_mask=attention_mask,
#             token_type_ids=token_type_ids,
#             position_ids=position_ids,
#             head_mask=head_mask,
#             inputs_embeds=inputs_embeds,
#         )

#         pooled_output = outputs[1]

#         pooled_output = self.dropout(pooled_output)
#         logits = self.classifier(pooled_output)

#         outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

#         if labels is not None:
#             if self.num_labels == 1:
#                 #  We are doing regression
#                 loss_fct = MSELoss()
#                 loss = loss_fct(logits.view(-1), labels.view(-1))
#             else:
# #                 loss_fct = CrossEntropyLoss()
#                 loss_fct = BCEWithLogitsLoss()
#                 loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
#             outputs = (loss,) + outputs

#         return outputs  # (loss), logits, (hidden_states), (attentions)